In [2]:
import aiohttp
import asyncio
import nest_asyncio
import base64
import os
import requests

# Apply nest_asyncio to allow nested event loops in Jupyter Notebook
nest_asyncio.apply()


In [3]:
def list_connections(admin_url):
    response = requests.get(f'{admin_url}/connections')
    if response.status_code == 200:
        connections = response.json()
        print("Connections:", connections)
        return connections
    else:
        print(f"Failed to list connections: {response.status_code}")
        print(response.text)
        return None
        
def read_and_encode_file(file_path):
    with open(file_path, 'rb') as file:
        file_content = file.read()
    encoded_content = base64.b64encode(file_content).decode('utf-8')
    return encoded_content

async def send_didcomm_message(admin_url, conn_id, message):
    async with aiohttp.ClientSession() as session:
        url = f'{admin_url}/connections/{conn_id}/send-message'
        payload = {
            "content": message
        }
        async with session.post(url, json=payload) as response:
            if response.status == 200:
                result = await response.json()
                print("Message sent successfully:")
                print(result)
            else:
                print(f"Failed to send message: {response.status}")
                print(await response.text())

async def main():
    # Define your ACA-Py admin API URL
    admin_url = 'http://localhost:8031'  # Replace with Faber's actual admin API URL

    # List connections to verify the connection ID
    connections = list_connections(admin_url)
    if not connections:
        return

    # Use a valid connection ID from the listed connections
    if 'results' in connections and len(connections['results']) > 0:
        conn_id = connections['results'][0]['connection_id']  # Use the first connection ID for example
    else:
        print("No connections found.")
        return

    # Path to the data folder containing files
    data_folder = 'data'

    # Read and encode all files in the folder
    encoded_files = []
    for file_name in os.listdir(data_folder):
        file_path = os.path.join(data_folder, file_name)
        if os.path.isfile(file_path):
            encoded_content = read_and_encode_file(file_path)
            encoded_files.append((file_name, encoded_content))

    # DIDComm message to send (with encoded file attachments)
    didcomm_message_template = {
        "type": "https://didcomm.org/file-transfer/1.0/message",
        "attachments": []
    }

    # Send each encoded file as an attachment in a DIDComm message repeatedly (e.g., 100 times)
    num_repeats = 1
    for _ in range(num_repeats):
        for file_name, encoded_content in encoded_files:
            didcomm_message = didcomm_message_template.copy()
            didcomm_message["attachments"] = [
                {
                    "filename": file_name,
                    "data": {
                        "base64": encoded_content
                    }
                }
            ]
            await send_didcomm_message(admin_url, conn_id, encoded_content)

# Run the main function
asyncio.run(main())

Connections: {'results': [{'state': 'active', 'created_at': '2024-05-30T06:56:03.263850Z', 'updated_at': '2024-05-30T06:56:03.447998Z', 'connection_id': '7901ceeb-35c3-4c73-bf50-adc8e41da91a', 'my_did': 'did:peer:4zQmcx9GY1Xz5MbYw9i4KBXb2T73NpWFJJxPx38t9y7bEfSS', 'their_did': 'did:peer:4zQmSkwv97NY7MbC5jA3gwt3b4nJxUwwuzfdkWvY9kBTGfho', 'their_label': 'faber.agent', 'their_role': 'inviter', 'connection_protocol': 'didexchange/1.1', 'rfc23_state': 'completed', 'invitation_key': '3uN8qMJGb7xQapfFBSfn9QboZBi5TxuURbF4tqCvgB2Q', 'invitation_msg_id': '43cbe95e-3922-41f9-b925-0f63c625247b', 'request_id': 'c03f7a4b-d80d-4933-9af1-69e097cc0900', 'accept': 'auto', 'invitation_mode': 'once'}]}
Message sent successfully:
{}
Message sent successfully:
{}
Failed to send message: 413
Maximum request body size 1048576 exceeded, actual body size 1048576
